In [1]:
# scikit learn 방식

In [2]:
import mlflow
mlflow.__version__

'3.1.0'

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


mlflow.set_tracking_uri('http://127.0.0.1:5000')

In [4]:
iris = load_iris()
x_train, x_test, y_train, y_test = train_test_split(
    iris.data, iris.target, test_size = 0.2, random_state=42
)

with mlflow.start_run():
    # 모델 학습
    clf = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=42)
    clf.fit(x_train, y_train)


    # 예측및평가
    y_pred = clf.predict(x_test)
    acc = accuracy_score(y_test, y_pred)

    # 메트릭, 모델 로깅
    mlflow.log_param('n_estimators', 100)
    mlflow.log_param('max_depth', 3)
    mlflow.log_metric('accuracy', acc)

    mlflow.sklearn.log_model(clf, 'model')
    print('logged model accuracy: ', acc)

2025/06/11 16:04:35 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/11 16:04:37 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/06/11 16:04:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


logged model accuracy:  1.0
🏃 View run legendary-carp-28 at: http://127.0.0.1:5000/#/experiments/0/runs/801ee05fbc8c43e59a91f586e5a22c0a
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0


In [5]:
import mlflow
import mlflow.sklearn
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from mlflow.models import infer_signature
from mlflow.tracking import MlflowClient

# MLflow 설정
mlflow.set_tracking_uri('http://127.0.0.1:5000')
mlflow.set_experiment("Iris-Sklearn-RandomSearch")

# 데이터 로드
iris = load_iris(as_frame=True)
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# 파이프라인 정의
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', RandomForestClassifier(random_state=42))
])

# 하이퍼파라미터 탐색
param_grid = {
    'clf__n_estimators': [50, 100, 150],
    'clf__max_depth': [2, 3, 4, 5, 6]
}

search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_grid,
    n_iter=5,
    cv=3,
    random_state=42
)
search.fit(X_train, y_train)

# 예시 입력/서명 추출
input_example = X_test[:5]
y_pred = search.predict(X_test)
signature = infer_signature(X_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

# MLflow Run
with mlflow.start_run(run_name="random_forest_search") as run:
    mlflow.log_params(search.best_params_)
    mlflow.log_metric("accuracy", accuracy)

    # 모델 저장
    model_info = mlflow.sklearn.log_model(
        sk_model=search.best_estimator_,
        artifact_path="model",
        input_example=input_example,
        signature=signature
    )

    # 모델 등록
    registered_model_name = "iris_rf_model"
    result = mlflow.register_model(
        model_uri=model_info.model_uri,
        name=registered_model_name
    )

    version = result.version
    print(f"모델 등록 완료: {registered_model_name}, version {version}")

# ---------------------------------------------------
# 모델 설명, 태그, 별칭 추가 (등록 이후 별도로 수행)
# ---------------------------------------------------
client = MlflowClient()

# 등록 모델 설명 업데이트
client.update_registered_model(
    name=registered_model_name,
    description="Random Forest 모델로 Iris 품종을 분류합니다. 전처리 포함된 sklearn 파이프라인 사용."
)

# 버전별 태그 추가
client.set_model_version_tag(registered_model_name, version, "accuracy", f"{accuracy:.4f}")
client.set_model_version_tag(registered_model_name, version, "cv", "3-fold")
client.set_model_version_tag(registered_model_name, version, "model_type", "RandomForest")

# 별칭 설정 (가장 최신 버전을 Production으로 지정)
client.set_registered_model_alias(
    name=registered_model_name,
    alias="production",
    version=version
)

print(f"설명, 태그, alias 설정 완료: version {version}")

2025/06/11 16:04:40 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/11 16:04:42 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
C:\Users\Admin\Desktop\LGSW\LGSW02\ml_dl\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Registered model 'iris_rf_model' already exists. Creating a new version of this model...
2025/06/11 16:04:42 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris_rf_model, version 2
Created version '2' of model 'iris_rf_model'.


모델 등록 완료: iris_rf_model, version 2
🏃 View run random_forest_search at: http://127.0.0.1:5000/#/experiments/370467540616997944/runs/a912c2c26934443f84131cfc2d1e53df
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/370467540616997944
설명, 태그, alias 설정 완료: version 2


# Pytorch

In [9]:
import mlflow
import mlflow.pytorch
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

# MLflow 설정
mlflow.set_tracking_uri('http://127.0.0.1:5000')
mlflow.set_experiment("Pytorch-SimpleCNN")

class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # [B, 32, 16, 16]
        x = self.pool(F.relu(self.conv2(x)))  # [B, 64, 8, 8]
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

# 데이터셋 가져오기
# CIFAR10
transform = transforms.Compose([
    # 입력사이즈, 다른 모델 사용 시 필수, 여기선 생략
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5))
])

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)

testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

2025/06/11 16:09:48 INFO mlflow.tracking.fluent: Experiment with name 'Pytorch-SimpleCNN' does not exist. Creating a new experiment.


In [10]:
device

device(type='cuda')

In [15]:
# MLflow 실험 기록
with mlflow.start_run():
    model = SimpleCNN().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)    

    mlflow.log_param('optimizer','Adam')
    mlflow.log_param('lr', 0.001)
    mlflow.log_param('epoches', 5)

    for epoch in range(5):
        model.train()
        running_loss = 0.0
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_loss = running_loss / len(trainloader)
        mlflow.log_metric('train_loss', avg_loss, step=epoch)
        print(f"Epock {epoch+1}, Loss: {avg_loss: .4f}")

        # 테스트 정확도 평갸
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for images, labels in testloader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = correct / total
        mlflow.log_metric('test_accuracy', accuracy)
        print(f"Test Accuracy: {accuracy:.4f}")
        mlflow.pytorch.log_model(model, name='model')

Epock 1, Loss:  1.3292


2025/06/11 16:23:01 WARNING mlflow.utils.requirements_utils: Found torch version (2.7.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.7.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


Test Accuracy: 0.6120


2025/06/11 16:23:08 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.22.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torchvision==0.22.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/06/11 16:23:08 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/06/11 16:23:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Epock 2, Loss:  0.9476


...

Epock 5, Loss:  0.5190


2025/06/11 16:24:36 WARNING mlflow.utils.requirements_utils: Found torch version (2.7.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.7.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


Test Accuracy: 0.7215


2025/06/11 16:24:42 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.22.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torchvision==0.22.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/06/11 16:24:42 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/06/11 16:24:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run puzzled-ant-746 at: http://127.0.0.1:5000/#/experiments/684382986429723580/runs/2ca2a19eaf3548cc901e432d01bcabaa
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/684382986429723580
